<a href="https://colab.research.google.com/github/thanksbinh/minBert/blob/main/minBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Nov 28 06:14:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.4/319.4 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
  

In [ ]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 384 # what is the maximum context length for predictions?
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 512
n_heads = 8 # --> every head is 512/8 = 64 dimensional
n_layers = 6
dropout = 0.1
max_documents = 1801350
# ------------

In [ ]:
from transformers import BertTokenizerFast
model_name = "bert-base-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
tokenizer.add_tokens(['\n'])

def stoi(text):
  """
  String to index.
  Converts a string to a list of token indices using the BERT tokenizer.
  """
  inputs = tokenizer(text, return_tensors="pt").to(device)
  return inputs.input_ids[0]


def itos(token_indices):
  """
  Index to string.
  Converts a list of token indices to a string using the BERT tokenizer.
  """
  # tokens = tokenizer.convert_ids_to_tokens(token_indices)
  # return tokenizer.convert_tokens_to_string(tokens, clean_up_tokenization_spaces=True)
  return tokenizer.decode(token_indices, clean_up_tokenization_spaces=True, skip_special_tokens=True)

def encode(s):
  # return stoi(s)
  sentences = s.split('.')
  encoded_sentences = []
  for sentence in sentences:
    encoded_sentences.append(stoi(sentence))
  # return encoded_sentences
  return torch.cat(encoded_sentences)

def decode(l):
  return itos(l)
  # decoded_sentences = []
  # for sentence in l:
  #   decoded_sentences.append(itos(sentence))
  # return '.'.join(decoded_sentences)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
ds[1]['text']

' = Valkyria Chronicles III = \n'

In [ ]:
# create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode(ds[1]['text']))
print(decode(encode(ds[1]['text'])))

tensor([  101, 28996,   134, 28996, 12226,  3781,  3464, 28996, 17758, 28996,
         2684, 28996,   134, 28996, 28996,   102])

 = 
 Valkyria 
 Chronicles 
 III 
 = 
 



In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = []
for i in range(max_documents):
  if ds[i]['text'] == '':
    continue
  data.append(torch.tensor(encode(ds[i]['text'][:block_size]), dtype=torch.long))
# data = torch.cat(data)
# print(data.shape, data.dtype)
print(len(data))
print(data[:5])

<ipython-input-9-bedb16793742>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.append(torch.tensor(encode(ds[i]['text'][:block_size]), dtype=torch.long))


1165029
[tensor([  101, 28996,   134, 28996, 12226,  3781,  3464, 28996, 17758, 28996,
         2684, 28996,   134, 28996, 28996,   102]), tensor([  101, 28996, 14895, 21006, 28996,  1185, 28996, 12226,  3781,  3464,
        28996,   124, 28996,   131, 28996,   133,  8362,  1377,   135, 28996,
        17758, 28996,   113, 28996,  1983, 28996,   131, 28996, 28996,   100,
        28996, 28996,   100, 28996,   100, 28996,   117, 28996,  4941, 28996,
          102,   101, 28996, 12226,  3781,  3464, 28996,  1104, 28996,  1103,
        28996,  2651,  2427, 28996,   124, 28996,   114, 28996,   117, 28996,
         3337, 28996,  2752, 28996,  1106, 28996,  1112, 28996, 12226,  3781,
         3464, 28996, 17758, 28996,  2684, 28996,  1796, 28996,  1999, 28996,
          117, 28996,  1110, 28996,   170, 28996, 12394, 28996,  1648, 28996,
          137,   118,   137, 28996,  1773, 28996,  1888, 28996,  1342, 28996,
         1872, 28996,  1118, 28996, 17979, 28996,  1105, 28996,  3957,   102,
   

In [ ]:
# vocab_size = 0
# for i in range(len(data)):
#   vocab_size = max(vocab_size, data[i].max().item())
# vocab_size += 1
# print(vocab_size)

vocab_size = 28997

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n]
val_data = data[n:]

In [ ]:
train_data[0][:block_size+1]

tensor([  101, 28996,   134, 28996, 12226,  3781,  3464, 28996, 17758, 28996,
         2684, 28996,   134, 28996, 28996,   102])

In [ ]:
from torch.nn import CrossEntropyLoss

def mask_tokens(inputs, mlm_probability=0.15):
    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, mlm_probability)

    # Create a mask where 15% of tokens will be masked
    mask = torch.bernoulli(probability_matrix).bool()

    # Make sure we don’t mask special tokens like [CLS], [SEP]
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    mask = mask & ~torch.tensor(special_tokens_mask, dtype=torch.bool)

    # 80% of the time, replace masked input tokens with [MASK]
    inputs[mask] = tokenizer.convert_tokens_to_ids("[MASK]")


    # 10% of the time, replace masked input tokens with random words
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long).to(device)
    ran = torch.rand(labels.shape) < 0.1
    inputs[mask & ran] = random_words[mask & ran]

    # 10% of the time, keep the original word (no change)

    # Return masked inputs and the labels (for calculating the loss)
    return inputs, labels

def get_batch(split):
    # generate a small batch of data of inputs, labels
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    batch_data = []
    for i in ix:
      len_batch = 0
      cur_batch = []
      j = i
      while len_batch < block_size:
        if j >= len(data):
          break
        cur_batch.append(data[j])
        len_batch += len(data[j])
        j += 1
      batch_data.append(torch.cat(cur_batch)[:block_size])
    batch_data = torch.stack(batch_data, dim=0)

    inputs, labels = mask_tokens(batch_data)
    inputs, labels = inputs.to(device), labels.to(device)
    return inputs, labels

def get_batch_gpt(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            inputs, labels = get_batch(split)
            logits, last_hidden_state = model(inputs)

            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
            losses[k] = loss.item()

        out[split] = losses.mean()
    model.train()
    return out

inputs, labels = get_batch('train')
print('inputs:')
print(inputs.shape)
print(inputs[0])
print('labels:')
print(labels.shape)
print(labels[0])

inputs:
torch.Size([64, 384])
tensor([  101, 28996,  1230, 28996, 15682, 28996,  2139,   103, 12721, 16310,
        28996,   103, 28996,  1140, 28996,   107, 28996,  4446,  3169, 28996,
          107, 28996,   782, 28996,  1167,   103,  1190, 28996,   170, 28996,
         1505, 28996,  1113, 28996,   103, 28996,  1271, 28996,   117, 28996,
         1290, 28996,  1122,   103,  1882, 28996,  1106, 28996,  4248,   103,
          103,   103, 17104, 28996,  1315, 28996,   102,   101, 28996,  1230,
        28996, 24912,   103,  1104, 28996,   103, 28996, 22233, 28996,   117,
        28996,  1137, 28996,  1148, 28996,  3853,   103,  1107,   103,  2855,
        28996,   117, 28996,  1108, 28996, 17047, 28996,   131, 28996,  1120,
        28996,  1655, 28996,  5729,   103, 28996,   103, 28996,  1118, 28996,
         2588,  1559, 28996,   103, 28996,   102,   101, 28996,  2038,  1757,
        28996,  1112,   103,  7140, 19511,   103, 28996, 25488,  2217,   103,
         1104, 28996,   103,   103

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        # self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attention_mask=None):
        B, T, C = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # compute attention score "affinities"
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

        # GPT
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # Apply attention mask if provided
        if attention_mask is not None:
            # attention_mask: (B, T) -> (B, 1, T) for broadcasting
            mask = attention_mask[:, None, :]
            wei = wei.masked_fill(mask == 0, float('-inf'))

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) # Prevent some nodes from communicating
        out = wei @ v
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attention_mask=None):
        out = torch.cat([h(x, attention_mask) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x, attention_mask=None):
        # Residual connection, clean flow
        x = x + self.sa(self.ln1(x), attention_mask)
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, y = inputs
        for module in self._modules.values():
            x = module(x, y)
        return x

In [ ]:
class Bert(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = SequentialEncoder(*[EncoderBlock(n_embed, n_heads=n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size) # for MLM

    def forward(self, idx, attention_mask=None, token_type_ids=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        token_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = token_embed + pos_embed # (B,T,C)
        x = self.blocks(x, attention_mask) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        last_hidden_state = x
        return logits, last_hidden_state

In [ ]:
# # Model from scratch
# bert_model = Bert()
# m = bert_model.to(device)

# inputs, labels = get_batch('train')

# logits, last_hidden_state = m(inputs)
# print(logits.shape)

# # Assume `prediction_logits` are the outputs from your model, and `labels` are the true token IDs
# loss_fct = CrossEntropyLoss()
# masked_lm_loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
# print(masked_lm_loss)

In [ ]:
# loading the saved model
import pickle

bert_model = pickle.load(open('bert_model_10000.sav', 'rb'))
m = bert_model.to(device)

In [ ]:
import pickle

prev_iters = 8500
max_iters = 5000
eval_interval = 500

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

# training loop
for iter in range(max_iters + 1):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss(m)
        print(f"step {prev_iters + iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # Add save point
        filename = 'bert_model_' + str(prev_iters + iter) + '.sav'
        pickle.dump(m, open(filename, 'wb'))

    # sample a batch of data
    inputs, labels = get_batch('train')

    # evaluate the loss
    logits, last_hidden_state = m(inputs)
    loss = F.cross_entropy(logits.view(-1, vocab_size), labels.view(-1))

    # zero the gradients from the previous iteration
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


NameError: name 'estimate_loss' is not defined


```
step 0: train loss 10.2286, val loss 10.2289
step 500: train loss 0.7812, val loss 0.7853
step 1000: train loss 0.7010, val loss 0.7052
step 1500: train loss 0.6475, val loss 0.6508
step 2000: train loss 0.5718, val loss 0.5745
step 2500: train loss 0.5263, val loss 0.5291
step 3000: train loss 0.4798, val loss 0.4825
step 3500: train loss 0.4333, val loss 0.4359
step 4000: train loss 0.4151, val loss 0.4178
step 4500: train loss 0.3995, val loss 0.4051
step 5000: train loss 0.3869, val loss 0.3896
step 5500: train loss 0.3774, val loss 0.3788
step 6000: train loss 0.3630, val loss 0.3667
step 6500: train loss 0.3495, val loss 0.3553
step 7000: train loss 0.3399, val loss 0.3460
step 7500: train loss 0.3289, val loss 0.3355
step 8000: train loss 0.3208, val loss 0.3259
step 8500: train loss 0.3148, val loss 0.3201
step 9000: train loss 0.3107, val loss 0.3157
step 9500: train loss 0.3041, val loss 0.3095
step 10000: train loss 0.3007, val loss 0.3048

```

In [ ]:
test_sentence = 'The quick brown fox jumps over the lazy dog'
inputs = stoi(test_sentence)
labels = inputs.clone()
inputs[-3] = 103
inputs = torch.unsqueeze(inputs, dim=0)
labels = torch.unsqueeze(labels, dim=0)

print(inputs)
print(decode(inputs[0]))
print(labels)
print(decode(labels[0]))

logits, last_hidden_state = m(inputs)
print(logits.shape, logits)

NameError: name 'stoi' is not defined

In [ ]:
logits_i = logits[:, -3, :] # becomes (B, C)
# apply softmax to get probabilities
probs = F.softmax(logits_i, dim=-1) # (B, C)
# sample from the distribution
print('probs', probs.shape, probs)
# idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
idx_next = torch.argmax(probs, dim=-1) # (B, 1)
# # append sampled index to the running sequence
print('idx_next', idx_next)

probs torch.Size([1, 28997]) tensor([[2.9492e-12, 3.8409e-12, 1.8883e-12,  ..., 4.6972e-12, 1.9590e-12,
         9.9874e-01]], grad_fn=<SoftmaxBackward0>)
idx_next tensor([28996])


In [ ]:
decoded_sentences = []
decoded_sentences.append(decode(idx_next))

print(decoded_sentences)

['\n']


In [ ]:
# class BERTForQuestionAnswering(nn.Module):
#     def __init__(self):
#         super(BERTForQuestionAnswering, self).__init__()

#         # Load the BERT model
#         self.bert = m

#         # Define the start position head
#         self.start_head = nn.Linear(vocab_size, 1)

#         # Define the end position head
#         self.end_head = nn.Linear(vocab_size, 1)

#         # Optional: Initialize weights
#         nn.init.normal_(self.start_head.weight, std=0.02)
#         nn.init.normal_(self.end_head.weight, std=0.02)

#     def forward(self, input_ids, attention_mask=None, token_type_ids=None):
#         # Get BERT outputs
#         outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

#         # Get the last hidden state (shape: [batch_size, sequence_length, hidden_size])
#         sequence_output = outputs.last_hidden_state
#         print(sequence_output.shape)

#         # Apply start head (shape: [batch_size, sequence_length, 1])
#         start_logits = self.start_head(sequence_output).squeeze(-1)
#         print(start_logits.shape)

#         # Apply end head (shape: [batch_size, sequence_length, 1])
#         end_logits = self.end_head(sequence_output).squeeze(-1)

#         return start_logits, end_logits


# model_qa = BERTForQuestionAnswering()
# m_qa = model_qa.to(device)

In [ ]:
# inputs, labels = get_batch('train')

# start_logits, end_logits = m_qa(inputs)
# print(start_logits.shape, end_logits.shape)

# loss_fct = CrossEntropyLoss()
# loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
# print(loss)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BertForQuestionAnswering(nn.Module):
    def __init__(self, bert_model):
        super(BertForQuestionAnswering, self).__init__()
        self.bert = bert_model  # Pretrained BERT model passed as an argument
        # hidden_size = bert_model.config.hidden_size  # Hidden size of BERT embeddings
        hidden_size = n_embed  # Hidden size of BERT embeddings

        # Add a linear layer for predicting start and end positions
        self.qa_outputs = nn.Linear(hidden_size, 2)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, start_positions=None, end_positions=None):
        # Forward pass through the BERT model to get hidden states

        logits, last_hidden_state = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = last_hidden_state  # [batch_size, sequence_length, hidden_size]

        # Apply the linear layer to get start and end logits
        logits = self.qa_outputs(sequence_output)  # [batch_size, sequence_length, 2]
        start_logits, end_logits = logits.split(1, dim=-1)  # Separate start and end logits
        start_logits = start_logits.squeeze(-1)  # [batch_size, sequence_length]
        end_logits = end_logits.squeeze(-1)      # [batch_size, sequence_length]

        # If start_positions and end_positions are provided, compute the loss
        if start_positions is not None and end_positions is not None:
            # Cross-entropy loss for start and end positions
            loss_fct = nn.CrossEntropyLoss()
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2
            return total_loss

        return start_logits, end_logits


In [ ]:
import torch
from transformers import BertTokenizerFast

# Load pre-trained BERT model and tokenizer for question answering
model_name = "bert-base-cased"
# pretrained_bert = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# tokenizer.add_tokens(['\n'])

In [ ]:
# model = BertForQuestionAnswering.from_pretrained(model_name)
qa_model = BertForQuestionAnswering(m)
qa_model.to(device)

BertForQuestionAnswering(
  (bert): Bert(
    (token_embedding_table): Embedding(28997, 512)
    (position_embedding_table): Embedding(384, 512)
    (blocks): SequentialEncoder(
      (0): EncoderBlock(
        (sa): MultiHeadAttention(
          (heads): ModuleList(
            (0-7): 8 x Head(
              (query): Linear(in_features=512, out_features=64, bias=False)
              (key): Linear(in_features=512, out_features=64, bias=False)
              (value): Linear(in_features=512, out_features=64, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ffwd): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.1, inplace=

In [ ]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load SQuAD v1.1 dataset
squad = load_dataset("squad")

# Tokenize the inputs for the model
def preprocess_data(batch):
    # Tokenize the batch of questions and contexts
    inputs = tokenizer(
        batch["question"],
        batch["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )
    offset_mapping = inputs.pop("offset_mapping")
    sequence_ids = inputs.sequence_ids()

    # Prepare start and end positions for each example in the batch
    start_positions = []
    end_positions = []

    for i in range(len(batch["answers"])):
        answers = batch["answers"][i]

        # Find start and end character positions
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])

        # Map character positions to token positions
        token_start_index = sequence_ids.index(1)
        token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Set token positions
        start_position, end_position = 0, 0
        for j, (offset_start, offset_end) in enumerate(offset_mapping[i]):
            if offset_start <= start_char < offset_end:
                start_position = j
            if offset_start < end_char <= offset_end:
                end_position = j

        start_positions.append(start_position)
        end_positions.append(end_position)

    # Add start and end positions to inputs
    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)

    return inputs

In [ ]:
# Apply preprocessing to the train and validation sets
train_data = squad["train"].map(preprocess_data, batched=True, remove_columns=squad["train"].column_names)

In [ ]:
val_data = squad["validation"].map(preprocess_data, batched=True, remove_columns=squad["validation"].column_names)

In [ ]:
from torch.utils.data import DataLoader, default_collate
from transformers import AdamW

def collate_fn(batch):
    # Stack input_ids, attention_mask, start_positions, and end_positions
    input_ids = torch.stack([torch.tensor(item["input_ids"]) for item in batch])
    attention_mask = torch.stack([torch.tensor(item["attention_mask"]) for item in batch])
    start_positions = torch.stack([torch.tensor(item["start_positions"]) for item in batch])
    end_positions = torch.stack([torch.tensor(item["end_positions"]) for item in batch])

    # Return a dictionary with tensors
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "start_positions": start_positions,
        "end_positions": end_positions
    }

In [ ]:
# Define DataLoader
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [ ]:
val_loader = DataLoader(val_data, batch_size=8, collate_fn=collate_fn)

In [ ]:
# Define optimizer
optimizer = AdamW(qa_model.parameters(), lr=3e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    qa_model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].squeeze().to(device)
        attention_mask = batch["attention_mask"].squeeze().to(device)
        start_positions = batch["start_positions"].squeeze().to(device)
        end_positions = batch["end_positions"].squeeze().to(device)

        optimizer.zero_grad()

        # Forward pass
        loss = qa_model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        # loss = outputs.total_loss
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

Epoch 1, Loss: 0.9517
Epoch 2, Loss: 0.7819


KeyboardInterrupt: 


Epoch 1/3, Loss: 3.8989

Epoch 2/3, Loss: 3.2813



Epoch 1, Loss: 3.5989

Epoch 2, Loss: 2.6745

Epoch 3, Loss: 2.3218

Epoch 4, Loss: 1.9987

Epoch 5, Loss: 1.6900

Epoch 6, Loss: 1.4086

Epoch 7, Loss: 1.1564

Epoch 8, Loss: 0.9517

Epoch 9, Loss: 0.7819

In [ ]:
pickle.dump(qa_model, open('qa_model_9.sav', 'wb'))

In [ ]:
import pickle

qa_model = pickle.load(open('qa_model.sav', 'rb'))
qa_model.to(device)

BertForQuestionAnswering(
  (bert): Bert(
    (token_embedding_table): Embedding(28997, 512)
    (position_embedding_table): Embedding(384, 512)
    (blocks): SequentialEncoder(
      (0): EncoderBlock(
        (sa): MultiHeadAttention(
          (heads): ModuleList(
            (0-7): 8 x Head(
              (query): Linear(in_features=512, out_features=64, bias=False)
              (key): Linear(in_features=512, out_features=64, bias=False)
              (value): Linear(in_features=512, out_features=64, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ffwd): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.1, inplace=

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00


In [ ]:
from transformers import default_data_collator
from tqdm import tqdm
import numpy as np
from evaluate import load

# Evaluation function
def evaluate_model(model, val_loader):
    qa_model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch["input_ids"].squeeze().to(device)
            attention_mask = batch["attention_mask"].squeeze().to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs

            # Predict start and end positions
            start_positions = torch.argmax(start_logits, dim=1)
            end_positions = torch.argmax(end_logits, dim=1)

            # Convert predictions and references to text
            for i in range(len(input_ids)):
                input_id = input_ids[i].tolist()
                start_pos = start_positions[i].item()
                end_pos = end_positions[i].item() + 1  # Inclusive

                # Decode predicted answer
                prediction = tokenizer.decode(input_id[start_pos:end_pos])
                predictions.append(prediction)

                # Reference answer
                answer =tokenizer.decode(input_id[batch["start_positions"][i]:batch["end_positions"][i] + 1])
                references.append(answer)

    # Compute metrics
    squad_metric = load("squad")
    predictions = [{'prediction_text': pred, 'id': str(i)} for i, pred in enumerate(predictions)]
    references = [{'answers': {'answer_start': [ref.index(char) for char in ref], 'text': [ref]}, 'id': str(i)} for i, ref in enumerate(references)]
    results = squad_metric.compute(predictions=predictions, references=references)
    return results


In [ ]:
# Run evaluation
scores = evaluate_model(qa_model, val_loader)
print(scores)

100%|██████████| 1322/1322 [01:46<00:00, 12.43it/s]


{'exact_match': 20.491958372753075, 'f1': 30.155657391843146}


{'exact_match': 7.653736991485336, 'f1': 13.292069194389137}

{'exact_match': 10.141911069063386, 'f1': 16.711711016874123}

{'exact_match': 19.87701040681173, 'f1': 28.703150757793537}

{'exact_match': 20.491958372753075, 'f1': 30.155657391843146}

In [ ]:
# from evaluate import load
# squad_metric = load("squad")
# predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22'}]
# references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
# results = squad_metric.compute(predictions=predictions, references=references)
# results